## Example Notebook

Examples used for EUFN Talk.


In [ ]:
%load_ext autoreload
%autoreload 2


from fibsem import utils, acquire, movement
from fibsem.structures import BeamType, Point, FibsemImage, FibsemStagePosition

import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
# connect to microscope
microscope, settings = utils.setup_session(manufacturer="Tescan", ip_address="localhost")


### Imaging Example I

Take an image with the Electron Beam

In [ ]:
from fibsem import acquire
from fibsem.structures import ImageSettings

# image settings
image_settings = ImageSettings(
    resolution = [1536, 1024], 
    dwell_time=1.0e-6,
    hfw=80-6,
    beam_type=BeamType.ELECTRON,
    autocontrast=True,
)

# acquire image
image = acquire.new_image(microscope, image_settings)

### Imaging Example II

Take an image with both beams at different hfws, and plot.


In [ ]:
settings.image.beam_type = BeamType.ION
settings.image.autocontrast = False
settings.image.save = True
settings.image.path = os.path.join(os.getcwd(), "demo", "imaging")
os.makedirs(settings.image.path, exist_ok=True)


settings.image.hfw = 80e-6

hfws = [80e-6, 150e-6, 400e-6, 900e-6]
for i, hfw in enumerate(hfws):

    settings.image.hfw = hfw
    settings.image.filename = f"hfws_{i:02d}"

    ib_image = acquire.new_image(microscope, settings.image)


### Loading Images, Reading Metadata

In [ ]:
# loading images

import glob
from fibsem.structures import FibsemImage

filenames = sorted(glob.glob(os.path.join(settings.image.path, "hfws*.tif")))

# plot wiht subplot
fig, axes = plt.subplots(1, len(filenames), figsize=(15, 5))
for fname in filenames:

    image = FibsemImage.load(fname)
    ax = axes[filenames.index(fname)]
    ax.imshow(image.data, cmap="gray")
    ax.set_title(f"{image.metadata.image_settings.filename} - {image.metadata.image_settings.hfw*1e6:.0f} um")
    ax.axis("off")

# subplot width
plt.tight_layout()

# hwidth
plt.subplots_adjust(wspace=0.01)

# save fig, dpi = 300
plt.savefig(os.path.join(settings.image.path, "hfws.png"), dpi=300)

plt.show()
    


### Stage Movement

Basic stage movement, and plotting.

In [ ]:
# image settings
settings.image.hfw = 150e-6
settings.image.save = True
settings.image.path = os.path.join(os.getcwd(), "demo", "movement")
os.makedirs(settings.image.path, exist_ok=True)

# acquire images with both beams
settings.image.filename = "pre-movement"
eb_image0, ib_image0 = acquire.take_reference_images(microscope, settings.image)

# move stage by 50um
move_position = FibsemStagePosition(x=50e-6)
microscope.move_stage_relative(move_position)

# acquire images with both beams
settings.image.filename = "post-movement"
eb_image1, ib_image1 = acquire.take_reference_images(microscope, settings.image)



# plot images
fig, axes = plt.subplots(2, 2, figsize=(10, 7))
axes[0][0].imshow(eb_image0.data, cmap="gray")
axes[0][0].set_title("Electron (pre-movement)")
# crosshair, centre, yellow
axes[0][0].plot(eb_image0.data.shape[1]/2, eb_image0.data.shape[0]/2, "y+", ms=20)
axes[0][1].imshow(ib_image0.data, cmap="gray")
axes[0][1].set_title("Ion (pre-movement)")
axes[0][1].plot(ib_image0.data.shape[1]/2, ib_image0.data.shape[0]/2, "y+", ms=20)

axes[1][0].imshow(eb_image1.data, cmap="gray")
axes[1][0].set_title("Electron (post-movement)")
# crosshair, centre, yellow
axes[1][0].plot(eb_image1.data.shape[1]/2, eb_image1.data.shape[0]/2, "y+", ms=20)
axes[1][1].imshow(ib_image1.data, cmap="gray")
axes[1][1].set_title("Ion (post-movement)")
axes[1][1].plot(ib_image1.data.shape[1]/2, ib_image1.data.shape[0]/2, "y+", ms=20)

# axes off
for ax in axes.flatten():
    ax.axis("off")

# wspace
plt.subplots_adjust(hspace=0.07, wspace=0.01)

# save fig, dpi = 300
plt.savefig(os.path.join(settings.image.path, "movement.png"), dpi=300)

plt.show()


## Scripting Examples

### Rotation Example

Rotate the stage, and take an image at each step (15deg).

In [ ]:
from fibsem import utils, acquire
from fibsem.structures import FibsemStagePosition
import os

# connect to microscope
microscope, settings = utils.setup_session(manufacturer="Tescan", ip_address="localhost")

# image settings
settings.image.save = True 
settings.image.path = os.path.join(os.getcwd(), "demo", "rotation")
os.makedirs(settings.image.path, exist_ok=True)

rot = 30    # rotation angle
for i in range(360//rot):
    
    # move stage relative
    microscope.move_stage_relative(FibsemStagePosition(r=np.deg2rad(rot)))

    # acquire images with both beams
    settings.image.filename = f"rot_{i:03d}"
    eb_image, ib_image = acquire.take_reference_images(microscope, settings.image)

## Tiling Example

Take a 3x3 tile scan with the ION beam and plot.

In [ ]:
from fibsem import utils, acquire
from fibsem.structures import FibsemStagePosition
import os

# connect to microscope
microscope, settings = utils.setup_session(manufacturer="Tescan", ip_address="localhost")

# image settings
settings.image.hfw = 80e-6
settings.image.resolution = [1024, 1024]
settings.image.beam_type = BeamType.ION
settings.image.save = True 
settings.image.path = os.path.join(os.getcwd(), "demo", "tile")
os.makedirs(settings.image.path, exist_ok=True)

# tile settings
dx, dy = settings.image.hfw, settings.image.hfw
nrows, ncols = 3, 3

# tile
initial_position = microscope.get_stage_position()
for i in range(nrows):

    # restore position
    microscope.move_stage_absolute(initial_position)
    # stable movement dy
    microscope.stable_move(dx=0, dy=dy*i, beam_type=BeamType.ION)

    for j in range(ncols):
    
        # stable movement dx
        microscope.stable_move(dx=dx, dy=0, beam_type=BeamType.ION)
        # acquire images with both beams
        settings.image.filename = f"tile_{i:03d}_{j:03d}"
        ib_image = acquire.new_image(microscope, settings.image)

In [ ]:
# plot tiles
import glob
filenames = sorted(glob.glob(os.path.join(settings.image.path, "tile*.tif")))


fig, axes = plt.subplots(nrows, ncols, figsize=(10, 10))
for fname in filenames:

    image = FibsemImage.load(fname)
    ax = axes[filenames.index(fname)//ncols][filenames.index(fname)%ncols]
    ax.imshow(image.data, cmap="gray")
    ax.axis("off")

# subplot width
plt.tight_layout()

# hwidth
plt.subplots_adjust(hspace=0.01,wspace=0.01)

# save fig, dpi = 300
plt.savefig(os.path.join(settings.image.path, "tiles.png"), dpi=300)

plt.show()

## Serialisation

Writing / Reading structures to disk

In [ ]:
from fibsem.structures import MicroscopeState
os.makedirs("demo/state", exist_ok=True)

# get state
print("\nCurrent microscope state:")
state = microscope.get_microscope_state()

# save to yaml
print("\nSerialise microscope state")
state_dict = state.to_dict()
utils.save_yaml("demo/state/state.yaml", state_dict)

# load from yaml
print("\nDeserialise microscope state")
state_dict = utils.load_yaml("demo/state/state.yaml")
state = MicroscopeState.from_dict(state_dict)

## Model Deployment

In [ ]:
from fibsem import utils, acquire
from fibsem.detection import detection
from fibsem.detection import detection
from fibsem.segmentation.model import load_model
from fibsem.structures import BeamType

# connect to microscope
microscope, settings = utils.setup_session(manufacturer="Demo", ip_address="localhost")

# model parameters
checkpoint = "autolamella-mega-latest.pt"

# load model
model = load_model(checkpoint=checkpoint) # type: ignore

# acquire image
settings.image.beam_type = BeamType.ION
image = acquire.new_image(microscope, settings.image)

# detect features
features = [detection.NeedleTip(), detection.LamellaCentre()] 
det = detection.detect_features(image=image.data, model=model, 
                                features=features,                      # type: ignore        
                                pixelsize=image.metadata.pixel_size.x)  # type: ignore 

# plot detection
detection.plot_detection(det)

In [ ]:
# model
%load_ext autoreload
%autoreload 2

from fibsem import utils, acquire
from fibsem.detection import detection
from fibsem.segmentation import model
from fibsem.detection import detection
from fibsem.detection.detection import DetectedFeatures
from fibsem.segmentation import model as fibsem_model
from fibsem.segmentation.model import load_model
from fibsem.structures import (
    BeamType,
    FibsemImage,
    Point,
)

# connect to microscope
microscope, settings = utils.setup_session(manufacturer="Demo", ip_address="localhost")

# model parameters
checkpoint = "autolamella-mega-latest.pt"

# load model
model = load_model(checkpoint=checkpoint) # type: ignore

# load image
image = FibsemImage.load("../fibsem/detection/test_image.tif")

# detect features
features = [detection.NeedleTip(), detection.LamellaCentre()] 
det = detection.detect_features(image=image.data, model=model, 
                                features=features, pixelsize=25e-9) # type: ignore 

# plot detection
detection.plot_detection(det)



### EUFN Live Example - Focus Stacking



In [ ]:
import os, logging
import matplotlib.pyplot as plt
import numpy as np
from fibsem import utils, acquire
from fibsem.structures import BeamType

# connect to microscope
microscope, settings = utils.setup_session(manufacturer="Thermo", ip_address="localhost") 

# setup
microscope.set("on", True, BeamType.ELECTRON )
microscope.set("on", True, BeamType.ION )

# image settings
settings.image.resolution = [6144, 4096]
settings.image.dwell_time = 200e-9
settings.image.hfw = 500e-6
settings.image.autocontrast = True
settings.image.save = True
settings.image.path = os.path.join(os.getcwd(), "demo", "eufn")
os.makedirs(settings.image.path, exist_ok=True)
# NB: no line integration

wds = np.arange(0.004, 0.0048, 50e-6)
images = []

logging.info(f"Acquiring {len(wds)} images")

for wd in wds:

    logging.info(f"Working distance: {wd:4e}")
    microscope.set("working_distance", wd, BeamType.ELECTRON)

    settings.image.filename = f"wd_{wd:.4e}".replace(".", "_")

    image = acquire.new_image(microscope, settings.image)

    images.append(image)
    logging.info(f"-"*200)

# lapacian pyramid
# TODO: stacking.py
# stacking.image_stacking(images)

In [ ]:

# plot images as grid
fig, axes = plt.subplots(len(images)//4, len(images)//4, figsize=(10, 10))

for i, image in enumerate(images):
    ax = axes[i//4, i%4]
    ax.imshow(image.data, cmap="gray")
    ax.set_title(f"WD: {wds[i]:.08f}")
    ax.axis("off")
# subplot width
plt.tight_layout()

# hwidth
plt.subplots_adjust(hspace=0.01,wspace=0.01)

plt.show()